# 03 — Memory with LangGraph 🧠

In this notebook, you’ll learn how to use **memory** in LangGraph — both **temporary** (in RAM) and **persistent** (saved to disk with SQLite).

Modern LangGraph handles memory through **checkpointers**, which automatically store and restore the graph’s state between runs.

We’ll build two examples:
- A temporary memory graph (cleared when the app stops)
- A persistent memory graph (saved using SQLite, reusable across sessions)

In [1]:
# --- Setup: Load environment and (optional) model ---
import os
from dotenv import load_dotenv, find_dotenv

_ = load_dotenv(find_dotenv())

openai_api_key = os.getenv("OPENAI_API_KEY")
groq_api_key = os.getenv("GROQ_API_KEY")
print("✅ Environment loaded.")

# Optional: define model if you want to use it later
# from langchain_openai import ChatOpenAI
# chat_model = ChatOpenAI(model="gpt-4o-mini")

✅ Environment loaded.


## 🧩 What is Memory in LangGraph?

LangGraph introduces the idea of **stateful graphs** — where each node can update or read from a **shared state**.  
To make that state persist across runs, LangGraph uses **Checkpointers**.

### Two main types of memory

| Type | Description | Use Case |
|------|--------------|-----------|
| **MemorySaver** | Stores state temporarily in RAM. | Fast experiments, prototypes. |
| **SqliteSaver** | Saves state permanently on disk. | Persistent agents, production apps. |

👉 The logic of your graph stays the same — only the checkpointer changes.

## 🧠 Example 1 — Temporary Memory (RAM)

Let’s start with a minimal example that stores state in memory.

When we run the graph multiple times **within the same session**, it remembers previous values.  
If you restart the notebook, the memory resets.

In [2]:
# Minimal in-memory counter with LangGraph (state kept in RAM)

from typing import TypedDict, Annotated
from operator import add

from langgraph.graph import StateGraph, START, END
from langgraph.checkpoint.memory import MemorySaver

# 1) State schema: 'count' accumulates by adding deltas
class State(TypedDict):
    count: Annotated[int, add]

# 2) Node: show next value; return the delta (+1)
def increment_node(state: State) -> State:
    current = state.get("count", 0) + 1
    print(f"Current count: {current}")
    return {"count": 1}  # delta to be added by the reducer

# 3) Build graph
graph = StateGraph(State)
graph.add_node("increment", increment_node)
graph.add_edge(START, "increment")
graph.add_edge("increment", END)

# 4) In-memory checkpointer (RAM only)
memory = MemorySaver()
app = graph.compile(checkpointer=memory)

# 5) Run multiple times with the same thread_id → state persists across invokes
thread_id = "session-001"

print(app.invoke({"count": 0}, config={"configurable": {"thread_id": thread_id}}))  # {'count': 1}
print(app.invoke({},          config={"configurable": {"thread_id": thread_id}}))    # {'count': 2}
print(app.invoke({},          config={"configurable": {"thread_id": thread_id}}))    # {'count': 3}

Current count: 1
{'count': 1}
Current count: 2
{'count': 2}
Current count: 3
{'count': 3}


### 🔍 Output (example)
```
Current count: 1
{'count': 1}
Current count: 2
{'count': 2}
Current count: 3
{'count': 3}
```

👉 Notice how the `count` variable **persists across invocations** as long as you use the same `thread_id`.

## 💾 Example 2 — Persistent Memory (SQLite)

For long-term persistence across restarts, use a **SQLite checkpointer**.  
This stores the graph’s state in a local database file (by default `memory.sqlite`).

You can close and reopen your notebook later — and the graph will continue where it left off.

In [4]:
from langgraph.checkpoint.sqlite import SqliteSaver

# 💾 Example 2 — Persistent Memory (SQLite)
# For long-term persistence across notebook restarts.

# Use context manager so the connection is properly opened/closed
with SqliteSaver.from_conn_string("memory.sqlite") as sqlite_memory:

    # 1️⃣ Compile the same graph, but now with SQLite checkpointer
    graph_persistent = graph.compile(checkpointer=sqlite_memory)

    # 2️⃣ Run multiple times (state persists between runs)
    thread_id = "persistent-session"

    for i in range(3):
        result = graph_persistent.invoke(
            {}, 
            config={"configurable": {"thread_id": thread_id}}
        )
        print(result)

Current count: 4
{'count': 4}
Current count: 5
{'count': 5}
Current count: 6
{'count': 6}


### ✅ What happens
- The first run initializes `count = 1`.
- The second and third runs continue from the previous value.
- If you **restart the notebook**, the counter still resumes from where it left off.

🧱 SQLite stores checkpoints in `memory.sqlite` (in your repo folder). You can inspect it with any SQLite viewer.

## ⚙️ How it works internally

Each time you run the graph:
1. LangGraph looks for an existing checkpoint for the same `thread_id`.
2. It restores the saved `state` from the last run.
3. It runs your nodes (which can update state values).
4. It saves the new state at the end of the execution.

That’s how you get **persistent stateful behavior** in agents and chains.

## 🔮 Practical use cases

- **Short-term memory:** keep a conversation or reasoning chain alive during an app session.
- **Persistent memory:** store progress or agent state between user sessions (e.g., saving task results, context summaries, or user preferences).
- **Hybrid setups:** use `MemorySaver` for scratchpad reasoning and `SqliteSaver` for long-term goals.

👉 You’ll use this same pattern later in multi-agent orchestration with **CrewAI** and **LangGraph** combined.

## ✅ Summary

| Type | Class | Persistence | Best For |
|------|--------|--------------|-----------|
| **MemorySaver** | `langgraph.checkpoint.memory` | Volatile (RAM) | Experiments, demos |
| **SqliteSaver** | `langgraph.checkpoint.sqlite` | Persistent (Disk) | Real apps, multi-session agents |

💡 Both use the same graph logic — only the **checkpointer** changes.